1. Définir la question à répondre

1.1 Combien de municipalités sont à risque de feu de forêt au Québec ?
    1.1.1 Quelle est la zone de feu de forêt au Québec ?
    1.1.2 Combien il y a de municipalité inclus dans cette zone ?

1.2 Identifier les biens et les infrastructures les plus exposés aux feux de forêt à l’échelle des communautés.

2. Définir les sources de données pour cette analyse.
    2.1 Identifier la zone de couverture du jeu de données
    2.2 Identifier le CRS



In [5]:
import geopandas as gpd
import requests, zipfile, io

In [6]:
# 2. Définir les sources de données pour cette analyse

# Municipalités
regionalUrl = 'https://diffusion.mern.gouv.qc.ca/diffusion/RGQ/Vectoriel/Theme/Regional/SDA_100k/SHP/BDAT(adm)_SHP.zip'
# Aéroport
airportsUrl = 'https://ws.mapserver.transports.gouv.qc.ca/swtq?service=wfs&version=2.0.0&request=getfeature&typename=ms:evenements&outfile=Aeroport&srsname=EPSG:4326&outputformat=geojson'
# villages autochtones
indigenousVillagesUrl = 'https://diffusion.mern.gouv.qc.ca/Diffusion/RGQ/Documentation/BDVA/Index_BDVA.zip'

firesUrl = 'https://diffusion.mffp.gouv.qc.ca/Diffusion/DonneeGratuite/Foret/PERTURBATIONS_NATURELLES/Feux_foret/FEUX_PROV_GPKG.zip'

# Empreintes de bâtiments
buildingFootPrint = 'https://diffusion.mern.gouv.qc.ca/diffusion/RGQ/Vectoriel/Theme/Local/Batiments/FGDB/QC_600019-54_Prevention_risques_sinistres_2019_fgdb.zip'

In [7]:
# Downloads the data sources, then format to DataFrame

# Cas de la source de données à partir de geojson
responseBody = requests.get(airportsUrl, stream=True).json()
airports = gpd.GeoDataFrame.from_features(responseBody['features'], crs="EPSG:4326")
# airports.head(10)

In [11]:
# Loading all raw data can take a moment
rawDataRelativePath = './rawData'
# Use Case from .zip file
response = requests.get(firesUrl)
zipFile = zipfile.ZipFile(io.BytesIO(response.content))
zipFile.extractall(path=rawDataRelativePath)


In [15]:
FEUX_PROV = 'FEUX_PROV.gpkg'
targetFile = "%s/%s" % (rawDataRelativePath, FEUX_PROV)
provincialFiresGeom = gpd.read_file(targetFile)
provincialFiresGeom.head()

,geoc_fmj,exercice,origine,an_origine,perturb,an_perturb,part_str,met_at_str,superficie,shape_length,shape_area,geometry
0,"-542638,44+891642,20",2000,BR,2000,NaN,NaN,NaN,MAJP,77.9,7889.024636,768342.785601,"MULTIPOLYGON (((-542763.350 892040.430, -54276..."
1,"-643830,30+485452,45",2000,BR,2000,NaN,NaN,NaN,MAJP,1.2,631.224487,11965.221500,"MULTIPOLYGON (((-643768.670 485505.720, -64377..."
2,"+173868,24+697820,37",2000,BR,2000,NaN,NaN,NaN,MAJP,25.4,4564.586167,250635.701400,"MULTIPOLYGON (((173949.200 698215.090, 173951...."
3,"-602121,43+549209,88",2000,BR,2000,NaN,NaN,NaN,MAJP,1.1,534.353529,11160.402850,"MULTIPOLYGON (((-602131.160 549282.970, -60213..."
4,"+351239,52+722405,16",2000,BR,2000,NaN,NaN,NaN,MAJP,32.0,3335.534767,316263.936250,"MULTIPOLYGON (((351640.030 722974.410, 351642...."
